# 행정안전부 도로명주소 API

공공 데이터 포털/통계청/기상청 등등 <br>

건물명으로부터 도로명주소 구하기 <br>
ID: beegramin PW: 소1!



In [6]:
import requests
from urllib.parse import quote 
# urlparse: url parsing을 위한 모듈 ,quote: 한글 인코딩
import pandas as pd


- urlparse <br>
url을 6요소로 분해

In [7]:
key_fd = open('../keys/roadmapkey.txt', mode='r', encoding='utf8')
roadapi_key = key_fd.read(100)
key_fd.close()

In [8]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'
# 변수집어넣는법
# https://www.juso.go.kr/addrlink/devAddrLinkRequestGuide.do?menu=roadApi

url = f'{road_url}?confmKey={roadapi_key}&currentPage=1&countPerPage=10&keyword=강서구청&resultType=json'

'https://www.juso.go.kr/addrlink/addrLinkApi.do?confmKey=U01TX0FVVEgyMDIwMTExOTEwMjMzNTExMDQzNzg=&currentPage=1&countPerPage=10&keyword=강서구청&resultType=json'

In [9]:
quote('강서구청')

'%EA%B0%95%EC%84%9C%EA%B5%AC%EC%B2%AD'

제대로 한글 인코딩을 하려면

In [10]:
url = f'{road_url}?confmKey={roadapi_key}&currentPage=1&countPerPage=10&keyword={quote("강서구청")}&resultType=json'

In [11]:
# 할 때마다 복사붙여넣기
results = requests.get(url).json()

In [12]:
# errorMessage는 신경안써도 됨
addr_list = results['results']['juso']
# addr_list

In [13]:
len(addr_list)

6

In [14]:
for addr in addr_list:
    print(addr['bdNm'],':',addr['roadAddr'])

강서구청 : 서울특별시 강서구 화곡로 302 (화곡동)
강서구청 : 서울특별시 강서구 화곡로44나길 72 (화곡동)
강서구청 : 부산광역시 강서구 낙동북로 477 (대저1동)
강서구청청소차고 : 서울특별시 강서구 남부순환로 208 (외발산동)
강서구청가양동별관 : 서울특별시 강서구 양천로59길 38 (가양동)
강서구청역 : 부산광역시 강서구 낙동북로 485 (대저1동)


In [15]:
districts = [
    '강서구청',
    '양천구청',
    '부천시청',
    '영등포구청',
    '김포시청'
]

In [22]:
road_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do'

road_address_dic = {}

for district in districts:
    url = f'{road_url}?confmKey={roadapi_key}&currentPage=1&countPerPage=10&keyword={quote(district)}&resultType=json'
    eachresult = requests.get(urlparse(url).geturl()).json()
    addr_list = eachresult['results']['juso']
    #for addr in addr_list:

    if addr_list[0]['bdNm'].find('부천시청') == 0:
        road_address_dic[addr_list[2]['bdNm']] = addr_list[2]['roadAddr']
        print(addr_list[2]['bdNm'],':',addr_list[2]['roadAddr'])
    else:
        road_address_dic[addr_list[0]['bdNm']] = addr_list[0]['roadAddr']
        print(addr_list[0]['bdNm'],':',addr_list[0]['roadAddr'])

강서구청 : 서울특별시 강서구 화곡로 302 (화곡동)
양천구청 : 서울특별시 양천구 목동동로 105 (신정동)
부천시청/부천시의회 : 경기도 부천시 길주로 210 (중동)
영등포구청 : 서울특별시 영등포구 당산로 123 (당산동3가)
김포시청 : 경기도 김포시 사우중로 1 (사우동)


In [18]:
road_address_dic

{'강서구청': '서울특별시 강서구 화곡로 302 (화곡동)',
 '양천구청': '서울특별시 양천구 목동동로 105 (신정동)',
 '부천시청/부천시의회': '경기도 부천시 길주로 210 (중동)',
 '영등포구청': '서울특별시 영등포구 당산로 123 (당산동3가)',
 '김포시청': '경기도 김포시 사우중로 1 (사우동)'}

In [19]:
df = pd.DataFrame([road_address_dic], index=['주소']).T
df.index.name = '구청'
df.reset_index('구청' ,inplace=True)
df

,구청,주소
0,강서구청,서울특별시 강서구 화곡로 302 (화곡동)
1,양천구청,서울특별시 양천구 목동동로 105 (신정동)
2,부천시청/부천시의회,경기도 부천시 길주로 210 (중동)
3,영등포구청,서울특별시 영등포구 당산로 123 (당산동3가)
4,김포시청,경기도 김포시 사우중로 1 (사우동)


In [20]:
# 인덱스 없이 저장하기
df.to_csv('address.csv',sep=',',encoding='utf8',index=False)

In [21]:
pd.read_csv('address.csv')

,구청,주소
0,강서구청,서울특별시 강서구 화곡로 302 (화곡동)
1,양천구청,서울특별시 양천구 목동동로 105 (신정동)
2,부천시청/부천시의회,경기도 부천시 길주로 210 (중동)
3,영등포구청,서울특별시 영등포구 당산로 123 (당산동3가)
4,김포시청,경기도 김포시 사우중로 1 (사우동)
